## Loading API Key

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
key = os.getenv("API_KEY")

## Imports

In [16]:
import requests
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import requests
from itertools import islice

## Function Definitions

In [17]:
def read_clans_range(file_name="clans_list.txt", start=0, end=100):
    with open(file_name, "r") as f:
        # Use islice for efficient range reading
        clans = list(islice(f, start, end))
    # Strip whitespace and return as a list of strings
    return [clan.strip() for clan in clans]


def get_clans_file(limit=100, minmembers=50, maxmembers=50, minscore=1000, key="", max_clans=3000, output_file="clans_list.txt"):
    all_clans = []
    after = None
    
    while len(all_clans) < max_clans:
        # Build the base URL
        url = f"https://api.clashroyale.com/v1/clans?minMembers={minmembers}&maxMembers={maxmembers}&minScore={minscore}&limit={limit}"
        
        # Add the 'after' marker if present
        if after:
            url += f"&after={after}"
        
        # Make the API call
        headers = {"Authorization": f"Bearer {key}"}
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            clans = data.get("items", [])
            paging = data.get("paging", {})
            
            # Add the clans to the master list
            all_clans.extend(clan["tag"] for clan in clans)
            
            # Get the next 'after' marker
            after = paging.get("cursors", {}).get("after")
            
            # Stop if there are no more pages
            if not after:
                print("Reached the last page.")
                break
            
            print(f"Retrieved {len(all_clans)} total clans so far...")
        
        else:
            print(f"Error retrieving clans: {response.status_code} - {response.text}")
            break
    
    # Save the clans to a file
    with open(output_file, "w") as f:
        for clan in all_clans[:max_clans]:
            f.write(clan + "\n")
    
    print(f"Saved {len(all_clans[:max_clans])} clans to {output_file}")

def call_api(url, key, verbose = 0):
    headers = {
        "Authorization": f"Bearer {key}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200 and verbose > 0:
        print("Success!")
    elif verbose > 0:
        print(f"Error: {response.status_code}")
    return response

def battles_to_dataframe(battles, acceptable_types = ["PvP", "pathOfLegend"]):
    rows = []
    for battle in battles:
        if battle['type'] in acceptable_types:
            player_won = battle["team"][0]["crowns"] > battle["opponent"][0]["crowns"]
            winlose = ["team", "opponent"] if player_won else ["opponent", "team"]
            row = {
                "battleTime": battle["battleTime"],
                "battleType": battle["type"],
                "battleMode": battle["gameMode"]["name"],
                "arena": battle["arena"]["id"],
                "winner": battle[winlose[0]][0]["tag"],
                "loser": battle[winlose[1]][0]["tag"],
            }
            labels = ['winner', 'loser']
            for k in range(2):
                for i, player in enumerate(battle[winlose[k]]):
                    for j, card in enumerate(player["cards"]):
                        row[f"{labels[k]}_card_{j+1}_id"] = card["id"]
                        row[f"{labels[k]}_card_{j+1}_level"] = card["level"]
                    tower_card = player["supportCards"][0]
                    row[f"{labels[k]}_tower_card_id"] = tower_card["id"]
                    row[f"{labels[k]}_tower_card_level"] = tower_card["level"]
            
            rows.append(row)
    
    df = pd.DataFrame(rows)
    id_cols = [col for col in df.columns if "_id" in col]
    level_cols = [col for col in df.columns if "_level" in col]
    df[id_cols] = df[id_cols].astype("int32")
    df[level_cols] = df[level_cols].astype("int8")
    
    return df

def get_player_battles(player_id, key):
    if player_id.startswith("#"):
        player_id = player_id[1:]
    url = f"https://api.clashroyale.com/v1/players/%23{player_id}/battlelog"
    return battles_to_dataframe(call_api(url, key).json())

def get_clans(limit = 100, minmembers = 50, minscore = 1000, key = key):
    clan_url = f"https://api.clashroyale.com/v1/clans?minMembers={minmembers}&minScore={minscore}&limit={limit}"
    clan_json = call_api(clan_url, key)
    clans = [i["tag"] for i in clan_json.json()["items"]]
    return clans

def get_members_from_clan(clan, key):
    members_url = f"https://api.clashroyale.com/v1/clans/%23{clan[1:]}/members"
    members_json = call_api(members_url, key)
    members = [i["tag"] for i in members_json.json()["items"]]
    return members

def get_full(file_name, num_clans = 100, key = key, clans_list = None, max_workers = 1):
    if not clans_list:
        clans = get_clans(num_clans, key=key)
    else:
        clans = clans_list
    print(f"Retrieved {len(clans)} clans")
    pd
    players = []
    for c in clans:
        players += get_members_from_clan(c, key)
    players = list(set(players))
    print(f"Retrieved {len(players)} players")
    if max_workers == 1:
        df = get_battles_sequential(players, key, file_name)
    else:
        df = get_battles_parallel(players, key, file_name, max_workers)
    df.to_csv(f"{file_name}.csv", index=False)
    print(f"Saved final df to {file_name}.csv")
    print("Done!")
    return df

def get_battles_sequential(players, key, file_name):
    df = pd.DataFrame()
    for p in range(len(players)):
        print(f"Retrieving battles for player {p}/{len(players)}     ", end="\r")
        try:
            player_df = get_player_battles(players[p], key)
            df = pd.concat([df, player_df])
        except Exception as e:
            print(f"Error retrieving battles for player {p}: {e}")
        if p + 1 %100 == 0:
            df.to_csv(f"{file_name}.csv", index=False)
            print(f"Saved {p} players to {file_name}.csv", end = "\r")
    print(f"Retrieving battles for player {p + 1}/{len(players)}")
    return df

def get_battles_parallel(players, key, file_name, max_workers):
    df_list = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(get_player_battles, player, key) for player in players]
        for i, future in enumerate(as_completed(futures)):
            try:
                player_df = future.result()
                if not player_df.empty:
                    df_list.append(player_df)
                if (i + 1) % 500 == 0:
                    pd.concat(df_list, ignore_index=True).to_csv(f"{file_name}.csv", index=False)
                    print(f"Saved {i + 1} players to {file_name}.csv", end="\r")
            except Exception as e:
                print(f"Error retrieving battles for player: {e}")
    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

## Testing API Key Functionality

In [17]:
my_id = "ULCYC0UU"
url = f"https://api.clashroyale.com/v1/players/%23{my_id}/battlelog"
url = "https://api.clashroyale.com/v1/clans?maxMembers=30&minMembers=20&minScore=1000&limit=3000"
x = call_api(url, key).json()
x

{'items': [{'tag': '#QG88V992',
   'name': '#1 SOLO WARRIOR',
   'type': 'inviteOnly',
   'badgeId': 16000081,
   'clanScore': 40441,
   'clanWarTrophies': 1483,
   'location': {'id': 57000120,
    'name': 'Italy',
    'isCountry': True,
    'countryCode': 'IT'},
   'requiredTrophies': 0,
   'donationsPerWeek': 548,
   'clanChestLevel': 1,
   'clanChestMaxLevel': 0,
   'members': 20},
  {'tag': '#R9LR9LLR',
   'name': '#阳光城市︱南昌',
   'type': 'open',
   'badgeId': 16000143,
   'clanScore': 45454,
   'clanWarTrophies': 1433,
   'location': {'id': 57000056,
    'name': 'China',
    'isCountry': True,
    'countryCode': 'CN'},
   'requiredTrophies': 0,
   'donationsPerWeek': 2155,
   'clanChestLevel': 1,
   'clanChestMaxLevel': 0,
   'members': 30},
  {'tag': '#R0V2R0VC',
   'name': '#阳光城市︱太原',
   'type': 'open',
   'badgeId': 16000143,
   'clanScore': 25263,
   'clanWarTrophies': 573,
   'location': {'id': 57000056,
    'name': 'China',
    'isCountry': True,
    'countryCode': 'CN'},
   '

In [204]:
my_id = "ULCYC0UU"
battles = get_player_battles(my_id, key)
battles.head(3)

,battleTime,battleType,battleMode,arena,winner,loser,winner_card_1_id,winner_card_1_level,winner_card_2_id,winner_card_2_level,...,loser_card_5_id,loser_card_5_level,loser_card_6_id,loser_card_6_level,loser_card_7_id,loser_card_7_level,loser_card_8_id,loser_card_8_level,loser_tower_card_id,loser_tower_card_level
0,20250508T032734.000Z,pathOfLegend,Ranked1v1_NewArena,54000114,#Q0UVVYLUC,#ULCYC0UU,26000047,11,26000064,11,...,26000021,9,26000041,11,27000006,11,26000052,9,159000000,11
1,20250508T030738.000Z,pathOfLegend,Ranked1v1_NewArena,54000114,#2002U9P0LC,#ULCYC0UU,26000024,11,26000064,11,...,26000021,9,26000041,11,27000006,11,26000052,9,159000000,11
2,20250508T030326.000Z,pathOfLegend,Ranked1v1_NewArena,54000114,#P8PRV2UGC,#ULCYC0UU,26000036,9,28000008,11,...,26000021,9,26000041,11,27000006,11,26000052,9,159000000,11


## Downloading Entire Dataset

In [57]:
clanstest = get_clans_file(limit=100, minmembers=48, maxmembers=48, minscore=1000, key=key, max_clans=3000, output_file="clans_list_48.txt")

Retrieved 100 total clans so far...
Retrieved 200 total clans so far...
Retrieved 300 total clans so far...
Retrieved 400 total clans so far...
Retrieved 500 total clans so far...
Retrieved 600 total clans so far...
Retrieved 700 total clans so far...
Retrieved 800 total clans so far...
Retrieved 900 total clans so far...
Reached the last page.
Saved 960 clans to clans_list_48.txt


In [32]:
import time
slice_num = 9

In [58]:
for i in range(10):
    print(f"Retrieving clans {i*100} to {i*100 + 100}")
    clans = read_clans_range(start=i*100, end=i*100+100, file_name="clans_list_48.txt")
    df = get_full(file_name = f'48_clash_battles_{i*100}_{i*100 + 100}', num_clans=100, key=key, clans_list=clans, max_workers = 8)
    time.sleep(5)

Retrieving clans 0 to 100
Retrieved 100 clans
Retrieved 4800 players
Saved final df to 48_clash_battles_0_100.csv.csv
Done!
Retrieving clans 100 to 200
Retrieved 100 clans
Retrieved 4800 players
Saved final df to 48_clash_battles_100_200.csv.csv
Done!
Retrieving clans 200 to 300
Retrieved 100 clans
Retrieved 4801 players
Saved final df to 48_clash_battles_200_300.csv.csv
Done!
Retrieving clans 300 to 400
Retrieved 100 clans
Retrieved 4800 players
Saved final df to 48_clash_battles_300_400.csv.csv
Done!
Retrieving clans 400 to 500
Retrieved 100 clans
Retrieved 4803 players
Saved final df to 48_clash_battles_400_500.csv.csv
Done!
Retrieving clans 500 to 600
Retrieved 100 clans
Retrieved 4801 players
Saved final df to 48_clash_battles_500_600.csv.csv
Done!
Retrieving clans 600 to 700
Retrieved 100 clans
Retrieved 4802 players
Saved final df to 48_clash_battles_600_700.csv.csv
Done!
Retrieving clans 700 to 800
Retrieved 100 clans
Retrieved 4800 players
Saved final df to 48_clash_battles_70

In [56]:
def check_clans_overlap(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        clans1 = set(f1.read().splitlines())
        clans2 = set(f2.read().splitlines())
        overlap = clans1.intersection(clans2)
        return overlap

overlap = check_clans_overlap('clans_list_49.txt', 'clans_list_48.txt')
if overlap:
    print("There are clans that appear in both files:")
    print(overlap)
else:
    print("No clans appear in both files.")

No clans appear in both files.


## Getting Card Dictionary for later

In [18]:
import json

fs = 'https://api.clashroyale.com/v1/cards'
y = call_api(fs, key).json()

card_mappings = {card['id']:card['name'] for card in y['items']}
tower_mappings = {card['id']:card['name'] for card in y['supportItems']}
card_mappings.update(tower_mappings)

json_data = json.dumps(card_mappings)

file_path = "./dicts/card_mappings.json"

with open(file_path, "w") as file:
    file.write(json_data)

In [23]:
file_path = "./dicts/card_info.json"

with open(file_path, "w") as file:
    json.dump(y, file)